# Loading Modules

In [ ]:
import gcsfs #module for google cloud connection
import os
import allel
import zarr
import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns
import h5py

Phase1 and Phase2 are different, either the outgroup alleles are different from phase2. They're based on phase 1 pass and REF and ALT alleles are different on the 2 phases. 
So the 1st step is find the positions shared from phase1 and phase2. After do that, apply these positions to phase2. Then map outgroup chromosomes.

### Loading datasets of phase1 and phase2

In [130]:
gcs_orig = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='cache')
gcs =  gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token=gcs_orig.session.credentials)

In [149]:
phase2path = os.path.join("ag1000g-release", "phase2.AR1", "variation", "main", "zarr", "biallelic", "ag1000g.phase2.ar1.pass.biallelic")

In [150]:
gcsmap2 = gcsfs.mapping.GCSMap(phase2path, gcs=gcs)

In [151]:
calldata_phase2= zarr.Group(gcsmap2, read_only=True)

In [144]:
phase1path = os.path.join("ag1000g-release", "phase1.AR3", "variation", "main", "zarr", "ag1000g.phase1.ar3.pass")

In [145]:
gcsmap1 = gcsfs.mapping.GCSMap(phase1path, gcs=gcs)

In [146]:
calldata_phase1= zarr.Group(gcsmap1, read_only=True)

-----------------------------------------------------

### Loading positions

In [92]:
pos_phase1 = allel.SortedIndex(calldata_phase1["2L"]["variants/POS"][:])
pos_phase1

0,1,2,3,4,...,10377275,10377276,10377277,10377278,10377279
44688,44691,44732,44736,44756,...,49356424,49356425,49356426,49356429,49356435


In [152]:
pos_phase2 = allel.SortedIndex(calldata_phase2["2L"]["variants/POS"])
pos_phase2

0,1,2,3,4,...,8906418,8906419,8906420,8906421,8906422
25050,51212,51214,51226,51245,...,49356421,49356424,49356425,49356426,49356429


Now I have to take only phase1 positions in phase2

In [96]:
loc1, loc2 = pos_phase2.locate_intersection(pos_phase1)
loc1, loc2

(array([False,  True,  True, ...,  True,  True,  True]),
 array([False, False, False, ...,  True,  True, False]))

In [99]:
pos_p2_sel = pos_phase2[loc1]
pos_p2_sel

0,1,2,3,4,...,7635919,7635920,7635921,7635922,7635923
51212,51214,51226,51245,51251,...,49356421,49356424,49356425,49356426,49356429


------------------------------

### Selecting same SNPs from christi dataset

In [100]:
calldata_out= h5py.File('/gcs/phase1.AR3/extras/outgroup_alleles.h5', mode='r')
calldata_out

<HDF5 file "outgroup_alleles.h5" (mode r)>

In [101]:
calldata_chr = calldata_out['2L']['chri']
calldata_chr[:], calldata_chr.shape

(array([b'.', b'.', b'.', ..., b'.', b'.', b'.'], dtype='|S1'), (10377280,))

In [104]:
sel_snp_chri = calldata_chr[loc2]
sel_snp_chri

array([b'.', b'.', b'.', ..., b'.', b'.', b'.'], dtype='|S1')

In [105]:
len(sel_snp_chri)

7635924

----------------------------

### Filtering Variants

In [106]:
variants_phase2 = allel.VariantChunkedTable(calldata_phase2["2L"]["variants"], 
                                     names=['POS', 'REF', 'ALT', 'DP', 'MQ', 'QD', 'numalt'],
                                     index='POS')
variants_phase2

<VariantChunkedTable shape=(8906423,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1'), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4'), ('numalt', '<i4')]
   nbytes=186.9M cbytes=-7 cratio=-27991615.1
   values=zarr.hierarchy.Group>

In [116]:
variants_phase2_filt = variants_phase2.compress(loc1)
variants_phase2_filt

<VariantChunkedTable shape=(7635924,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1'), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4'), ('numalt', '<i4')]
   nbytes=160.2M cbytes=74.2M cratio=2.2
   values=allel.chunked.storage_zarr.ZarrTable>

### Create arrays for mapping

In [119]:
phase2_ref = variants_phase2_filt["REF"][:]
phase2_ref, phase2_ref.shape

(array([b'T', b'G', b'A', ..., b'G', b'G', b'A'], dtype='|S1'), (7635924,))

In [120]:
phase2_alt = variants_phase2_filt["ALT"][:]
phase2_alt, phase2_alt.shape

(array([b'A', b'C', b'G', ..., b'T', b'T', b'G'], dtype='|S1'), (7635924,))

In [122]:
mapping = allel.create_allele_mapping(phase2_ref, phase2_alt, sel_snp_chri)
mapping

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       ...,
       [-1, -1],
       [-1, -1],
       [-1, -1]], dtype=int8)

------------------------------

In [125]:
chri_allele_count = allel.AlleleCountsArray(mapping)
chri_allele_count

<AlleleCountsArray shape=(7635924, 2) dtype=int8>
- -
- -
- -
...
- -
- -
- -

---------------------------------------------

### Phase2 allele count subset

In [1]:
metadata = pd.read_csv("samples.meta.txt", sep="\t")
pop_select = metadata.population.isin({'BFgam', 'GAgam', 'AOcol'}).values
pop_subset = metadata[pop_select]

NameError: name 'pd' is not defined

In [154]:
g = calldata_phase2["2L"]["calldata/GT"]

----------------------------------------------------------------

#### Sandbox

In [81]:
variants_phase1 = allel.VariantChunkedTable(calldata_phase1["2L"]["variants"], 
                                     names=['POS', 'REF', 'ALT', 'DP', 'MQ', 'QD', 'numalt'],
                                     index='POS')
variants_phase1

<VariantChunkedTable shape=(10377280,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1', (3,)), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4'), ('numalt', '<i4')]
   nbytes=237.5M cbytes=-7 cratio=-35579245.7
   values=zarr.hierarchy.Group>

In [9]:
calldata_phase2= zarr.Group(gcsmap2, read_only=True)

In [10]:
variants_phase2 = allel.VariantChunkedTable(calldata_phase2["2L"]["variants"], 
                                     names=['POS', 'REF', 'ALT', 'DP', 'MQ', 'QD', 'numalt'],
                                     index='POS')
variants_phase2

<VariantChunkedTable shape=(8906423,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1'), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4'), ('numalt', '<i4')]
   nbytes=186.9M cbytes=-7 cratio=-27991615.1
   values=zarr.hierarchy.Group>

In [60]:
calldata_out= h5py.File('/gcs/phase1.AR3/extras/outgroup_alleles.h5', mode='r')
calldata_out

<HDF5 file "outgroup_alleles.h5" (mode r)>

In [61]:
calldata_out.keys()

<KeysViewHDF5 ['2L', '2R', '3L', '3R', 'X']>

In [62]:
calldata_out['2L'].keys()

<KeysViewHDF5 ['arab', 'chri', 'epir', 'mela', 'meru', 'quad']>

In [63]:
calldata_out['2L']['chri'][1000_000:1000_010]

array([b'C', b'G', b'C', b'C', b'C', b'C', b'G', b'C', b'C', b'T'],
      dtype='|S1')

In [64]:
calldata_chr = calldata_out['2L']['chri']
calldata_chr[:], calldata_chr.shape

(array([b'.', b'.', b'.', ..., b'.', b'.', b'.'], dtype='|S1'), (10377280,))

In [65]:
calldata_out_count= h5py.File('/gcs/phase1.AR3/extras/outgroup_allele_counts.h5', mode='r')
calldata_out_count

<HDF5 file "outgroup_allele_counts.h5" (mode r)>

In [66]:
calldata_out_count.keys()

<KeysViewHDF5 ['2L', '2R', '3L', '3R', 'X']>

In [67]:
calldata_out_count['2L'].keys()

<KeysViewHDF5 ['arab', 'chri', 'epir', 'mela', 'meru', 'quad']>

In [68]:
calldata_out_count['2L']['chri'][1000_000:1000_010]

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]], dtype=int32)

In [69]:
calldata_chr_count = calldata_out_count['2L']['chri']
calldata_chr_count[:], calldata_chr_count.shape

(array([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]], dtype=int32), (10377280, 4))

In [26]:
b = np.asarray(calldata_chr)

In [41]:
asd = b.tolist()

In [45]:
a = list(filter(lambda x: x!= b'.', asd))

In [47]:
len(a)

7793733

In [94]:
phase2_ref = calldata_phase2["2L"]["variants"]["REF"][:]
phase2_ref, phase2_ref.shape

(array([b'G', b'T', b'G', ..., b'G', b'G', b'A'], dtype='|S1'), (8906423,))

In [95]:
phase2_alt = calldata_phase2["2L"]["variants"]["ALT"][:]
phase2_alt, phase2_alt.shape

(array([b'T', b'A', b'C', ..., b'T', b'T', b'G'], dtype='|S1'), (8906423,))

In [96]:
allel.create_allele_mapping(phase2_ref, phase2_alt, calldata_chr)

ValueError: arrays do not have matching length for dimension 0